## Using **Personas** to Simulate Hallux Data


### This example has two goals

* To assign personas to the different entities in the Hallux database either randomly or based on existing data.

* To use the assigned personas to predict an entity's behavoior in order to create new data.



### The example starts by importing the required libraries and creating dictionaries


In [12]:

# import libraries

from os import getenv
import pandas as pd
from datetime import *; from dateutil.relativedelta import *
import pymssql
from sqlalchemy import create_engine


# Personas dictionary (defined in Hallux tables Person_type and Persona)

Personas = {
    "Album" : ("Bomb","Non-Certified","Gold","Platinum","Multi Platinum","Diamond"), 
    "Band" : ("Bar Band","Cover","Tribute","1 hit wonder","Popular","Cult Following","Megaband"),
    "Customer" : ("Occasioanl","Discount","Loyal"),
    "Employee" : ("Bad","Good","Overachiever"),
    "Experience" : ("Bad","Good","Wonderful"),
    "Fan" : ("Casual","Fan","Fanatic"),
    "Music" : ("Popular", "Regional","Seasonal","Specialty","Nostalgic"),
    "Musician" : ("Amateur","Studio","Backup","Lead","Superstar"),
    "Performance" : ("Bad","Good","Epic"),
    "Person" : ("Student","Young Adult","Mature"),
    "Song" : ("Bad","Good","Popular","Hit"),
    "Video" : ("Bad","Good","Viral")
}

# Hallux entity types dictionary with value list (id column, persona type)

Entities = {
    "Agent"   : ("Agent_Id","Employee"),
    "Album" : ("Album_Id","Album"), 
    "Band" : ("Band_Id","Band"),
    "Customer" : ("Customer_Id","Customer"),
    "Follower" : ("Follower_Id","Fan"),
    "Genre" : ("Genre_Id","Music"), 
    "Member" : ("Member_Id","Musician"),
    "Order"   : ("Order_Id","Experience"),
    "Performance" : ("Performance_Id","Performance"),
    "Playlist" : ("Playlist_Id",""),
    "Producer"   : ("Producer_Id","Employee"),
    "Profile" : ("Profile_Id","Person"),
    "Song" : ("Song_Id","Song"),
    "Stream"   : ("Stream_Id","Experience"),
    "Video" : ("Video_Id","Video"),
}



### Determine an individual entity's persona

The set_persona function will update the database with an entities persona 

*any note*  


In [13]:

def set_persona (a_entity_type, a_entity_id, a_persona_type ):   
    
    my_persona_id = -1
    
    return my_persona_id


### Get an individual entity's persona

The get_persona function will return an entity's persona 

*any note*  

In [14]:

def get_persona (a_entity_type, a_entity_id, a_persona_type):   
    
    my_persona_id = -1
    
    return my_persona_id


###  Determine Persona 

The my_persona function will determine the persona for an entity

*any note* 

In [15]:


def my_personas ( a_entity_type, a_entity_id, a_persona_type ):   

    
    my_persona_id = -1
    
    return my_persona_id





###  Assign Personas 

The assign_personas function will loop thru all the members of a specific Hallux entity and assign them personas 

*This is where the hard work is done!*  

In [26]:

def assign_personas ( a_entity_type, a_persona_type ):   

    
    ret_val = -1
    
    # loop thru the entities and assign each a persona
    # determine_persona
    # set_persona
    
    return ret_val




### Simulate Follower Data

This fuction will determine which bands the specified customer profile will follow, if any 

*any note*  

In [22]:

def sim_follower_data (a_start_date, a_end_date):   
    
    ret_val = 0
    return ret_val


### Simulate Playlist Data

This fuction will determine which playlists the specified customer profile will create, if any 

*any note*  

In [23]:

def sim_playlist_data (a_start_date, a_end_date):   
    
    ret_val = 0
    return ret_val



### Simuate Streaming Data

This fuction will determine which songs the specified customer profile will stream, if any 

*any note*  

In [24]:

def sim_streaming_data (a_start_date, a_end_date):   
    
    ret_val = 0
    return ret_val




### Simuate Data

This fuction calls the appropriate simulation function based on the entitity type, otherwise return -1



In [20]:

def sim_data ( a_entity_type, a_start_date, a_end_date ):   

    ret_val = -1
    
    if a_entity_type == 'Follower':
        ret_val = sim_follower_data (a_start_date, a_end_date )
        
    if a_entity_type == 'Playlist':
        ret_val = sim_playlist_data (a_start_date, a_end_date )
        
    if a_entity_type == 'Stream':
        ret_val = sim_streaming_data (a_start_date, a_end_date )
        
    
    return ret_val



### Here we go...

* Initalization, database engine 

* Assign personas to Hallux entities 

* Generate the data for the specified date range!


In [27]:


hallux_svr = getenv("halluxsvr")
hallux_usr = getenv("halluxusr")
hallux_psd = getenv("halluxpsd")
hallux_db  = getenv("halluxdb")


# for use in creating a dataframe
conn_string = f"mssql+pyodbc://{hallux_usr}:{hallux_psd}@{hallux_svr}/{hallux_db}?driver=SQL+Server+Native+Client+11.0&TrustServerCertificate=yes"
eng = create_engine(conn_string, fast_executemany=True)

# used for executing dynamic sql
conn   = pymssql.connect(hallux_svr, hallux_usr, hallux_psd, hallux_db)


# STEP 1 - loop thru the entities and assign the appropriate persona

for curr_entity, curr_value in Entities.items():
    ret = assign_personas (curr_entity, curr_value[1])
    print(curr_entity, ret)


# STEP 2 - Loop thru the entities and Simulate data 

start_date = date(2023, 1, 1)
end_date   = date(2023, 12, 31)  

for curr_entity, curr_value in Entities.items():
    ret = sim_data (curr_entity, start_date, end_date)
    print(curr_entity, ret)


conn.close()
eng.dispose()



Agent -1
Album -1
Band -1
Customer -1
Follower -1
Genre -1
Member -1
Order -1
Performance -1
Playlist -1
Producer -1
Profile -1
Song -1
Stream -1
Video -1
Agent -1
Album -1
Band -1
Customer -1
Follower 0
Genre -1
Member -1
Order -1
Performance -1
Playlist 0
Producer -1
Profile -1
Song -1
Stream 0
Video -1
